In [1]:
import gmaps
import gmaps.datasets
import gmaps.geojson_geometries
import json
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
gmaps.configure(api_key="AI...") # Your Google API key
with open('region.json') as f:
     geometry = json.load(f)

In [3]:
latlong = pd.read_csv('tc.csv')
latlong.sort_values(by=['Region'],inplace=True)
dn = pd.read_csv('./datasets/nights_2010-2017.csv',parse_dates=['date'], index_col='date',usecols=[*range(1, 15)])
dftrain = dn[(dn.index>='2015-01-01')&(dn.index<'2015-02-01')]
dftrain = dftrain.T
dftrain.sort_index(inplace=True)
dftrain.columns = ['Nights']
dftrain.index = latlong.Region
dftrain= dftrain.reset_index()
dftrain.sort_values(by=['Region'],inplace=True)
tcdf = pd.merge(latlong, dftrain, on='Region', how='left')
x = tcdf['Nights'].values.reshape(-1, 1) #returns a numpy array
min_max_scaler = MinMaxScaler(feature_range=(1, 13))
x_scaled = min_max_scaler.fit_transform(x)
tcdf['Magnitude'] = x_scaled
locations = tcdf[['Latitude', 'Longitude']]
weights = tcdf['Magnitude']
fig = gmaps.figure()
geojson_layer = gmaps.geojson_layer(geometry)
fig.add_layer(geojson_layer)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights,  point_radius=80))
fig

Figure(layout=FigureLayout(height='420px'))